<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#include <GL/glew.h>
#include <GLFW/glfw3.h>
#include <cuda_runtime.h>
#include <cuda_gl_interop.h>
#include <iostream>

const int width = 800;
const int height = 600;

// CUDA カーネル関数
__global__ void kernel(float* ptr, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = y * width + x;

    if (x < width && y < height) {
        // 単純な色データを生成
        ptr[4 * idx + 0] = (float)x / width; // R
        ptr[4 * idx + 1] = (float)y / height; // G
        ptr[4 * idx + 2] = 0.0f; // B
        ptr[4 * idx + 3] = 1.0f; // A
    }
}

int main() {
    // GLFW を初期化
    if (!glfwInit()) {
        std::cerr << "GLFW 初期化に失敗しました" << std::endl;
        return -1;
    }

    // ウィンドウを作成
    GLFWwindow* window = glfwCreateWindow(width, height, "CUDA OpenGL Interop", NULL, NULL);
    if (!window) {
        std::cerr << "ウィンドウ作成に失敗しました" << std::endl;
        glfwTerminate();
        return -1;
    }

    glfwMakeContextCurrent(window);

    // GLEW を初期化
    if (glewInit() != GLEW_OK) {
        std::cerr << "GLEW 初期化に失敗しました" << std::endl;
        return -1;
    }

    // OpenGL バッファオブジェクトを作成
    GLuint vbo;
    glGenBuffers(1, &vbo);
    glBindBuffer(GL_ARRAY_BUFFER, vbo);
    glBufferData(GL_ARRAY_BUFFER, width * height * 4 * sizeof(float), NULL, GL_DYNAMIC_DRAW);

    // CUDA-OpenGL リソースを登録
    cudaGraphicsResource* cuda_vbo_resource;
    cudaGraphicsGLRegisterBuffer(&cuda_vbo_resource, vbo, cudaGraphicsMapFlagsNone);

    // ページロックメモリを確保
    float* host_ptr;
    cudaMallocHost((void**)&host_ptr, width * height * 4 * sizeof(float));

    // CUDA カーネル起動のための設定
    dim3 block(16, 16);
    dim3 grid((width + block.x - 1) / block.x, (height + block.y - 1) / block.y);

    while (!glfwWindowShouldClose(window)) {
        // CUDA リソースをマッピング
        cudaGraphicsMapResources(1, &cuda_vbo_resource, 0);
        float* d_ptr;
        size_t num_bytes;
        cudaGraphicsResourceGetMappedPointer((void**)&d_ptr, &num_bytes, cuda_vbo_resource);

        // CUDA カーネルを起動
        kernel<<<grid, block>>>(d_ptr, width, height);
        cudaDeviceSynchronize();

        // リソースをアンマップ
        cudaGraphicsUnmapResources(1, &cuda_vbo_resource, 0);

        // OpenGL レンダリング
        glClear(GL_COLOR_BUFFER_BIT);
        glBindBuffer(GL_ARRAY_BUFFER, vbo);
        glVertexPointer(4, GL_FLOAT, 0, 0);
        glEnableClientState(GL_VERTEX_ARRAY);
        glDrawArrays(GL_POINTS, 0, width * height);
        glDisableClientState(GL_VERTEX_ARRAY);

        // バッファをスワップ
        glfwSwapBuffers(window);
        glfwPollEvents();
    }

    // メモリを解放
    cudaFreeHost(host_ptr);
    cudaGraphicsUnregisterResource(cuda_vbo_resource);
    glDeleteBuffers(1, &vbo);

    // GLFW を終了
    glfwDestroyWindow(window);
    glfwTerminate();

    return 0;
}

In [ ]:

#include <GL/glut.h>
#include <cuda_runtime.h>
#include <cuda_gl_interop.h>
#include <iostream>

GLuint texture;
cudaGraphicsResource* cudaResource;
unsigned char* devPtr;
int width = 512;
int height = 512;

__global__ void kernel(unsigned char* ptr, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int offset = x + y * width;
        ptr[offset * 4 + 0] = 255;  // Red
        ptr[offset * 4 + 1] = 0;    // Green
        ptr[offset * 4 + 2] = 0;    // Blue
        ptr[offset * 4 + 3] = 255;  // Alpha
    }
}

void createTexture() {
    glGenTextures(1, &texture);
    glBindTexture(GL_TEXTURE_2D, texture);

    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP);

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, width, height, 0, GL_RGBA, GL_UNSIGNED_BYTE, nullptr);
    cudaGraphicsGLRegisterImage(&cudaResource, texture, GL_TEXTURE_2D, cudaGraphicsMapFlagsWriteDiscard);
}

void render() {
    cudaGraphicsMapResources(1, &cudaResource, 0);
    cudaArray* array;
    cudaGraphicsSubResourceGetMappedArray(&array, cudaResource, 0, 0);

    cudaMemcpyToArray(array, 0, 0, devPtr, width * height * 4 * sizeof(unsigned char), cudaMemcpyDeviceToDevice);
    cudaGraphicsUnmapResources(1, &cudaResource, 0);

    glClear(GL_COLOR_BUFFER_BIT);
    glEnable(GL_TEXTURE_2D);
    glBindTexture(GL_TEXTURE_2D, texture);

    glBegin(GL_QUADS);
    glTexCoord2f(0.0f, 0.0f); glVertex2f(-1.0f, -1.0f);
    glTexCoord2f(1.0f, 0.0f); glVertex2f(1.0f, -1.0f);
    glTexCoord2f(1.0f, 1.0f); glVertex2f(1.0f, 1.0f);
    glTexCoord2f(0.0f, 1.0f); glVertex2f(-1.0f, 1.0f);
    glEnd();

    glDisable(GL_TEXTURE_2D);
    glutSwapBuffers();
}

void update() {
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);
    kernel<<<gridSize, blockSize>>>(devPtr, width, height);

    render();
    glutPostRedisplay();
}

void cleanup() {
    cudaGraphicsUnregisterResource(cudaResource);
    cudaFree(devPtr);
    glDeleteTextures(1, &texture);
}

int main(int argc, char** argv) {
    glutInit(&argc, argv);
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGBA);
    glutInitWindowSize(width, height);
    glutCreateWindow("CUDA OpenGL Interop");

    glewInit();

    createTexture();

    cudaMalloc((void**)&devPtr, width * height * 4 * sizeof(unsigned char));

    glutDisplayFunc(render);
    glutIdleFunc(update);
    glutCloseFunc(cleanup);

    glutMainLoop();

    return 0;
}